In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib qt5

In [3]:
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import MaxNLocator

In [4]:
import pickle as pk

In [5]:
%load_ext autoreload
%autoreload 2

# Custom tools

In [6]:
import sys

In [7]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.posargb as gb
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.process.filtering import box_filter
from posarutils.process.filtering import filter2

In [8]:
day0 = "2019_02_22"
dir0 = f"/home/pleroy/DATA/DIADEM/lff/Bizona_{day0}/Raw_data" 
pre0 = "Volume_sol"
nbDir0 = 12

day1 = "2019_03_04"
dir1 = "/home/pleroy/DATA/DIADEM/lff/Bizona/" + day1
pre1 = "sand_trunk_vol"
nbDir1 = 11

RxTx = ["21", "31", "32", "41", "42", "43"]

# Load images

In [9]:
def getMinMaxdB(img):
    mindB = np.amin( 20 * np.log10( np.abs( img ) ) )
    maxdB = np.amax( 20 * np.log10( np.abs( img ) ) )
    return mindB, maxdB

In [10]:
filename = dir0 + f"/{day0}_3D_mono.dat"
with open(filename, 'rb') as file:
    (geomMono0, imgMono0) = pk.load( file )
    
filename = dir0 + f"/{day0}_3D_bi.dat"
with open(filename, 'rb') as file:
    (geomBi0, imgBi0) = pk.load( file )

filename = dir1 + f"/{day1}_3D_mono.dat"
with open(filename, 'rb') as file:
    (geomMono1, imgMono1) = pk.load( file )

filename = dir1 + f"/{day1}_3D_bi.dat"
with open(filename, 'rb') as file:
    (geomBi1, imgBi1) = pk.load( file )

In [11]:
minMono0, maxMono0 = getMinMaxdB( imgMono0 )
minMono1, maxMono1 = getMinMaxdB( imgMono1 )
minBi0, maxBi0     = getMinMaxdB( imgBi0 )
minBi1, maxBi1     = getMinMaxdB( imgBi1 )

/home/pleroy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log10
  
/home/pleroy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
print( f"Mono0 {minMono0:.1f} {maxMono0:.1f}\nMono1 {minMono1:.1f} {maxMono1:.1f}\n"
      + f"Bi0 {minBi0:.1f} {maxBi0:.1f}\nBi1 {minBi1:.1f} {maxBi1:.1f}" )

Mono0 -112.5 32.3
Mono1 -95.4 48.1
Bi0 -inf 56.3
Bi1 -inf 68.3


In [13]:
geomBi0, geomBi1, geomMono0, geomMono1

((-1.5, 1.5, 0.01, 0, 7, 0.01, -0.5, 1.5, 0.01),
 (-1.5, 1.5, 0.01, 0, 7, 0.01, -0.5, 1.5, 0.01),
 (-1.5, 1.5, 0.01, 0, 7, 0.01, -0.5, 1.5, 0.01),
 (-1.5, 1.5, 0.01, 0, 7, 0.01, -0.5, 1.5, 0.01))

In [14]:
(xMin, xMax, dx, yMin, yMax, dy, zMin, zMax, dz) = geomBi0
nbX = int( (xMax-xMin) / dx ) + 1
nbY = int( (yMax-yMin) / dy ) + 1
nbZ = int( (zMax-zMin) / dz ) + 1
extent_yx = [yMin, yMax, xMin, xMax]
extent_yz = [yMin, yMax, zMin, zMax]
extent_xz = [xMin, xMax, zMin, zMax]
x = np.arange(xMin,xMax+1e-9,dx)
y = np.arange(yMin,yMax+1e-9,dy)
z = np.arange(zMin,zMax+1e-9,dz)

# zCut

In [15]:
def plotMosaicComparison(mono0, mono1, bi0, bi1, vmin, vmax, extent, origin='upper'):

    fig = plt.figure()
    grid = AxesGrid(fig, 111,  # similar to subplot(111)
                    nrows_ncols=(2, 2),
                    axes_pad=0.0,
                    share_all=True,
                    label_mode="all",
                    cbar_location="top",
                    cbar_mode="single",
                    )

    toPlot = 20 * np.log10( np.abs( np.squeeze( mono0 ) ) )
    im = grid[0].imshow(toPlot, cmap='jet', extent=extent, origin=origin, vmin=vmin, vmax=vmax)
    grid[0].set_ylabel("monostatic")

    toPlot = 20 * np.log10( np.abs( np.squeeze( mono1 ) ) )
    im = grid[1].imshow(toPlot, cmap='jet', extent=extent, origin=origin, vmin=vmin, vmax=vmax)
    grid[1].set_ylabel("monostatic")
    grid[1].yaxis.set_ticks_position('right')
    grid[1].yaxis.set_label_position('right')

    toPlot = 20 * np.log10( np.abs( np.squeeze( bi0 ) ) )
    im = grid[2].imshow(toPlot, cmap='jet', extent=extent, origin=origin, vmin=vmin, vmax=vmax)
    grid[2].set_ylabel("bistatic")

    toPlot = 20 * np.log10( np.abs( np.squeeze( bi1 ) ) )
    im = grid[3].imshow(toPlot, cmap='jet', extent=extent, origin=origin, vmin=vmin, vmax=vmax)
    grid[3].set_ylabel("bistatic")
    grid[3].yaxis.set_ticks_position('right')
    grid[3].yaxis.set_label_position('right')

    grid.cbar_axes[0].colorbar(im)

    for ax in grid:
            ax.xaxis.set_major_locator(MaxNLocator(prune='upper', integer=True))
            ax.yaxis.set_major_locator(MaxNLocator(prune='upper', integer=True))
            
    return fig, grid

In [84]:
zCut   = -0.03
iz0 = np.where((z<zCut+dz) & (z>=zCut))[0]

In [85]:
fig, ax = plt.subplots(1,1)
toPlot = 10 * np.log10( np.abs( np.squeeze(imgMono1[:,:,iz0]) ) )
im = ax.imshow(toPlot, origin='lower', cmap='jet', extent=extent_yx)
dia.addColorBar( im, ax, aspect="equal" )
ax.invert_yaxis()
ax.set_xlim( 1, 6 )
#im.set_clim(-30,20)
title = f"{day1} {pre1}\ncut at z={zCut:.2f}"
ax.set_title(title)

Text(0.5, 1.0, '2019_03_04 sand_trunk_vol\ncut at z=-0.03')

In [90]:
2.2**0.5

1.4832396974191326

In [73]:
fig, ax = plt.subplots(1,1)
toPlot = 10 * np.log10( np.abs( np.squeeze(imgMono0[:,:,iz0]) ) )
im = ax.imshow(toPlot, origin='lower', cmap='jet', extent=extent_yx)
dia.addColorBar( im, ax, aspect="equal" )
ax.invert_yaxis()
ax.set_xlim( 1, 6 )
im.set_clim(-30,15)

In [59]:
fig, ax = plt.subplots(1,1)
toPlot = 10 * np.log10( np.abs( np.squeeze(imgBi0[:,:,iz0]) ) )
im = ax.imshow(toPlot, origin='lower', cmap='jet', extent=extent_yx)
dia.addColorBar( im, ax, aspect="equal" )

In [ ]:
fig, ax = plt.subplots(1,1)
toPlot = 10 * np.log10( np.abs( np.squeeze(imgBi1[:,:,iz0]) ) )
im = ax.imshow(toPlot, origin='lower', cmap='jet', extent=extent_yx)
dia.addColorBar( im, ax, aspect="equal" )

In [ ]:
vmin=-20
vmax=50
fig, grid = plotMosaicComparison( imgMono0[:,:,iz0], imgMono1[:,:,iz0], imgBi0[:,:,iz0], imgBi1[:,:,iz0],
                                 vmin, vmax, extent_yx, origin='lower' )
title = f"{day0} {pre0}\n{day1} {pre1}\ncut at z={zCut:.2f}"
fig.suptitle(title)

# xCut

In [21]:
xCut = -0.3
ix0 = np.where((x<xCut+dx) & (x>=xCut))[0]

In [22]:
vmin=-20
vmax=50
fig, grid = plotMosaicComparison( imgMono0[ix0,:,:].T, imgMono1[ix0,:,:].T, imgBi0[ix0,:,:].T, imgBi1[ix0,:,:].T,
                                 vmin, vmax, extent_yz, origin='lower' )
title = f"{day0} {pre0}\n{day1} {pre1}\ncut at x={xCut:.2f}"
fig.suptitle(title)

Text(0.5, 0.98, '2019_02_22 Volume_sol\n2019_03_04 sand_trunk_vol\ncut at x=-0.30')

# yCut

In [ ]:
yCut = 3.5
iy0 = np.where((y<xCut+dy) & (y>=yCut))[0]

In [ ]:
vmin=-20
vmax=50
fig, grid = plotMosaicComparison( imgMono0[:,iy0,:].T, imgMono1[:,iy0,:].T, imgBi0[:,iy0,:].T, imgBi1[:,iy0,:].T,
                                 vmin, vmax, extent_yz, origin='lower' )
title = f"{day0} {pre0}\n{day1} {pre1}\ncut at x={xCut:.2f}"
fig.suptitle(title)

# incoherent sum

## monostatic

### along z

In [38]:
zCutStart = -0.5
zCutStop = 1.5
izStart = np.where( (z<zCutStart+dz) & (z>=zCutStart) )[0][0]
izStop  = np.where( (z<zCutStop+dz)  & (z>=zCutStop) )[0][0]
izStart, izStop

(0, 200)

In [39]:
toSum = np.abs( imgMono1 )**2
imgSum = np.sum( toSum, axis=2 )

In [42]:
toPlot = 10 * np.log10( imgSum )
fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yx )
dia.addColorBar( im, ax, aspect="equal" )
title = f"{day0} {pre0}\nmonostatic sum images"
ax.set_title(title)
ax.invert_yaxis()
ax.set_xlim( 1, 6 )

(1, 6)

### along x

In [43]:
xCutStart = -1.5
xCutStop = 1.5
ixStart = np.where( (x<xCutStart+dx) & (x>=xCutStart) )[0][0]
ixStop  = np.where( (x<xCutStop+dx)  & (x>=xCutStop) )[0][0]
ixStart, ixStop

(0, 300)

In [44]:
toSum = np.abs( imgMono0[ixStart:ixStop, :, :] )**2
monoSum0 = np.sum( toSum, axis=0 )
toSum = np.abs( imgMono1[ixStart:ixStop, :, :] )**2
monoSum1 = np.sum( toSum, axis=0 )

In [50]:
toPlot = 10 * np.log10( monoSum1 )
fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot.T, cmap='jet', origin='lower', extent=extent_yz )
dia.addColorBar( im, ax, aspect="equal" )
title = f"{day0} {pre0}\nmonostatic sum images"
ax.set_title(title)
ax.set_xlim( 1, 6 )

(1, 6)

In [48]:
fig, ax = plt.subplots(2,1)

toPlot = 10 * np.log10( monoSum0 )
im = ax[0].imshow(toPlot.T, cmap='jet', origin='lower', extent=extent_yz, vmin=0, vmax=50 )
dia.addColorBar( im, ax[0], aspect="equal" )
title = f"{day0} {pre0} monostatic sum images x={xCutStart:.2f} .. {xCutStop:.2f}"
ax[0].set_title(title)

toPlot = 10 * np.log10( monoSum1 )
im = ax[1].imshow(toPlot.T, cmap='jet', origin='lower', extent=extent_yz, vmin=0, vmax=50 )
dia.addColorBar( im, ax[1], aspect="equal" )
title = f"{day1} {pre1} monostatic sum images x={xCutStart:.2f} .. {xCutStop:.2f}"
ax[1].set_title(title)

Text(0.5, 1.0, '2019_03_04 sand_trunk_vol monostatic sum images x=-1.50 .. 1.50')

## bistatic

### along x

In [ ]:
xCutStart = -0.1
xCutStop = 0.1
ixStart = np.where( (x<xCutStart+dx) & (x>=xCutStart) )[0][0]
ixStop  = np.where( (x<xCutStop+dx)  & (x>=xCutStop) )[0][0]
ixStart, ixStop

In [ ]:
toSum = np.abs( imgBi0[ixStart:ixStop, :, :] )**2
biSum0 = np.sum( toSum, axis=0 )
toSum = np.abs( imgBi1[ixStart:ixStop, :, :] )**2
biSum1 = np.sum( toSum, axis=0 )

In [ ]:
fig, ax = plt.subplots(2,1)

toPlot = 10 * np.log10( biSum0 )
im = ax[0].imshow(toPlot.T, cmap='jet', origin='lower', extent=extent_yz, vmin=0, vmax=70 )
dia.addColorBar( im, ax[0], aspect="equal" )
title = f"{day0} {pre0}\nbistatic sum images x={xCutStart:.2f} .. {xCutStop:.2f}"
ax[0].set_title(title)

toPlot = 10 * np.log10( biSum1 )
im = ax[1].imshow(toPlot.T, cmap='jet', origin='lower', extent=extent_yz, vmin=0, vmax=70 )
dia.addColorBar( im, ax[1], aspect="equal" )
title = f"{day1} {pre1}\nbistatic sum images x={xCutStart:.2f} .. {xCutStop:.2f}"
ax[1].set_title(title)

# xCut

In [ ]:
hRef   = 3.500
xCut   = 0.000
with open(base_path + f"/{date}_hRef{hRef:.3f}_xCut{xCut:.3f}.dat", 'rb') as file:
    img_h_zCut = pk.load(file)

In [ ]:
fig, grid = plotImgMosaic(img_h_zCut[hRef], extent=extent_yz, origin='lower', T=1)
title = f"{date} xCut = {xCut}"
fig.suptitle(title)

# Tomography

## Monostatic

In [ ]:
imgSum  = np.zeros( (nbY, nbZ), dtype=complex)
imgSumC = np.zeros( (nbY, nbZ), dtype=complex)
amp     = np.zeros( (nbY, nbZ), dtype=complex)
idxx = idx0_x
aver = 0

for height in heights:
    z3 = heights[height]
    for ij in ["21", "31", "32"]:
        imgSum +=         img_h[z3][ij][idxx,:,:]
        amp    += np.abs( img_h[z3][ij][idxx,:,:] )
        aver += 1
imgSum = imgSum / aver
min_imgSum = np.amin( 20 * np.log10( np.abs( imgSum ) ) )
max_imgSum = np.amax( 20 * np.log10( np.abs( imgSum ) ) )
xSum = x[idxx]
print(f"x = {xSum:.3f}, aver = {aver}, {min_imgSum:.1f} {max_imgSum:.1f}")

In [ ]:
#imgSumNorm = imgSum / amp
toPlot = 20 * np.log10( np.abs( imgSum ).T )
fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yz, vmin=vmin, vmax=vmax)
dia.addColorBar( im, ax, aspect="equal" )
title = f"{date} {prefix}\nmonostatic x = {x[idx0_x]:.3f}"
ax.set_title(title)

## Bistatic

In [ ]:
biz_imgSum  = np.zeros( (nbY, nbZ), dtype=complex)
biz_imgSumC = np.zeros( (nbY, nbZ), dtype=complex)
biz_amp     = np.zeros( (nbY, nbZ), dtype=complex)
idxx = idx0_x
aver = 0

for idx in heights:
    for ij in ["41", "42", "43"]:
        z3 = heights[idx]
        if z3 != heights['1']:
            if ij != "41":
                biz_imgSum +=         img_h[z3][ij][idxx,:,:]
                biz_amp    += np.abs( img_h[z3][ij][idxx,:,:] )
                aver+=1
        else:
            biz_imgSum +=         img_h[z3][ij][idxx,:,:]
            biz_amp    += np.abs( img_h[z3][ij][idxx,:,:] )
            aver+=1
biz_imgSum = biz_imgSum / aver
min_biz_imgSum = np.amin( 20 * np.log10( np.abs( biz_imgSum ) ) )
max_biz_imgSum = np.amax( 20 * np.log10( np.abs( biz_imgSum ) ) )
xBizSum = x[idxx]
print(f"x = {xBizSum:.3f}, aver = {aver}, min = {min_biz_imgSum:.1f}, max = {max_biz_imgSum:.1f}")

In [ ]:
#biz_imgSumNorm = biz_imgSum / biz_amp
toPlot = 20 * np.log10( np.abs(biz_imgSum).T )
fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yz, vmin=vmin, vmax=vmax )
dia.addColorBar( im, ax, aspect="equal" )
title = f"{date} {prefix}\nbistatic x = {x[idx0_x]:.2f}"
ax.set_title(title)

## Plot Mono Bi 2D Tomo

In [ ]:
fig, grid0, grid1 = dia.plotImgMosaicMonoBi2DTomo( img_h_zCut[3.41]["21"], img_h_zCut[3.41]["41"], 
                                                  imgSum.T, biz_imgSum.T, 
                                                  extent_yx, extent_yz, vmin=vmin, vmax=vmax )
fig.suptitle(date + f" x={xSum:.2f} z={zCut}")

## Monostatic incoherent sum (along x)

In [ ]:
amp_i = np.zeros( (nbY, nbZ), dtype=float )
aver = 0

for xn in range(nbX):
    imgSum  = np.zeros( (nbY, nbZ), dtype=complex )
    # compute the coherent sum for the considered xn
    for idx in heights:
        for ij in ["21", "31", "32"]:
            z3 = heights[idx]
            imgSum += img_h[z3][ij][xn,:,:]
            aver+=1
    # compute the non coherent sum
    amp_i = amp_i + imgSum ** 2
amp_i = (amp_i / aver)**0.5
min_amp_i = np.amin( 20 * np.log10( np.abs( amp_i ) ) )
max_amp_i = np.amax( 20 * np.log10( np.abs( amp_i ) ) )
print(f"aver = {aver}, {min_amp_i:.1f}dB {max_amp_i:.1f}dB")

In [ ]:
toPlot = 20 * np.log10( np.abs(amp_i).T )

fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yz, vmin=vmin, vmax=vmax)
dia.addColorBar( im, ax, aspect="equal" )
title = f"{date} monostatic\nsum intensities along x from {xMin} to {xMax}, dx = {dx}"
ax.set_title(title)

 ## Bistatic incoherent sum (along x)

In [ ]:
biz_amp_i = np.zeros( (nbY, nbZ), dtype=float )
aver = 0

for xn in range(nbX):
    biz_imgSum  = np.zeros( (nbY, nbZ), dtype=complex )
    # compute the coherent sum for the considered xn
    for idx in heights:
        for ij in ["41", "42", "43"]:
            z3 = heights[idx]
            if z3 != 3.5:
                if ij != "41":
                    biz_imgSum += img_h[z3][ij][xn,:,:]
                    aver+=1
            else:
                biz_imgSum += img_h[z3][ij][xn,:,:]
                aver+=1
    # compute the non coherent sum
    biz_amp_i = biz_amp_i + biz_imgSum ** 2
biz_amp_i = (biz_amp_i / aver)**0.5
min_biz_amp_i = np.amin( 20 * np.log10( np.abs( biz_amp_i ) ) )
max_biz_amp_i = np.amax( 20 * np.log10( np.abs( biz_amp_i ) ) )
print(f"aver = {aver}, {min_biz_amp_i:.1f} {max_biz_amp_i:.1f}")

In [ ]:
toPlot = 20 * np.log10( np.abs(biz_amp_i).T )

fig, ax = plt.subplots(1,1)
im = ax.imshow(toPlot, cmap='jet', origin='lower', extent=extent_yz, vmin=vmin, vmax=vmax)
dia.addColorBar( im, ax, aspect="equal" )
title = f"{date} bistatic\nsum intensities along x from {xMin} to {xMax}, dx = {dx}"
ax.set_title(title)

## Plot bistatic and monostatic

In [ ]:
fig, grid = dia.plotImgMosaicMonoBi( imgSum.T, biz_imgSum.T, extent_yz, origin='lower',vmin=20,vmax=65 )
title = date + f"yz cut at {x[idx0_x]:.3f}"
fig.suptitle(title)